In [1]:
import requests
import json
import pandas as pd
import numpy as np
from time import sleep
from datetime import datetime, timezone
from datetime import time

import matplotlib.pyplot as plt
import matplotlib

from utils import get_block_number_from_timestamp

In [2]:
df_timestamp_block_origin = pd.read_csv('./daily_data/timestamp_block.csv')
print("Origin date fetch (UTC):", datetime.utcfromtimestamp(df_timestamp_block_origin['timestamp'].iloc[-1]))
print("---------------------------------------")

origin_timestamp = df_timestamp_block_origin['timestamp'].iloc[-1]
dt = datetime.now()
last_timestamp = int(datetime(
    year=dt.year, month=dt.month, day=dt.day, 
    hour=0, minute=0, second=0, microsecond=0, 
    tzinfo=timezone.utc
).timestamp())

interval = 86400 # 1 day
df_timestamp_block = df_timestamp_block_origin

for timestamp in range(origin_timestamp, last_timestamp+1, interval):
    block = get_block_number_from_timestamp(timestamp)
    values = [{
        "timestamp": timestamp,
        "block": block
    }]
    df_timestamp_block = df_timestamp_block.append(values)
    sleep(0.25)

# not sure why got many duplicated, just use drop
df_timestamp_block.drop_duplicates(keep='first', inplace=True) 

# save
df_timestamp_block.reset_index(inplace = True, drop = True)
df_timestamp_block.to_csv('./daily_data/timestamp_block.csv', index=False)

print("Last date fetch (UTC):", datetime.utcfromtimestamp(df_timestamp_block['timestamp'].iloc[-1]))

Origin date fetch (UTC): 2022-08-09 00:00:00
---------------------------------------
Last date fetch (UTC): 2022-08-14 00:00:00


In [3]:
df_fres =  pd.read_csv('./daily_data/new_subgraph/fres.csv')

base_url = "https://v4.subgraph.polygon.oceanprotocol.com"
route = "/subgraphs/name/oceanprotocol/ocean-subgraph"
url = base_url + route
query = \
f"""
{{fixedRateExchanges(
    first: 1000, 
    # block: {{number: 30077156}}
    ) 
    {{
    active
    datatokenBalance
    price
    totalSwapValue
    datatoken {{
      cap
      holderCount
      orderCount
      symbol
      name
      publishMarketFeeAmount
      nft {{
        name
        transferable
        symbol
        owner
      }}
    }}
    
  }}}}
    """
headers = {"Content-Type": "application/json"}
payload = json.dumps({"query": query})
response = requests.request("POST", url, headers=headers, data=payload)

data = json.loads(response.text)
df_data = pd.json_normalize(data['data']['fixedRateExchanges'])

dt = datetime.now()
now_timestamp = int(datetime(
    year=dt.year, month=dt.month, day=dt.day, 
    hour=dt.hour, minute=dt.minute, second=dt.second, microsecond=0, 
    tzinfo=timezone.utc
).timestamp())
df_data["timestamp"] = now_timestamp

df_fres = pd.concat([df_fres, df_data], ignore_index=True, sort=False)
# df_fres = df_data

df_fres.drop_duplicates(keep='first', inplace=True,subset=['datatoken.symbol', 'datatoken.name','timestamp'])
df_fres.reset_index(inplace = True, drop = True)
df_fres.to_csv('./daily_data/new_subgraph/fres.csv', index=False)

df_fres

,active,datatokenBalance,price,totalSwapValue,datatoken.cap,datatoken.holderCount,datatoken.orderCount,datatoken.symbol,datatoken.name,datatoken.publishMarketFeeAmount,datatoken.nft.name,datatoken.nft.transferable,datatoken.nft.symbol,datatoken.nft.owner,timestamp
0,True,0,1,0,1157920892373161954235709850086879000000000000...,0,0,ADROYS-75,Adroit Oyster Token,0,Acentrik Asset Management,True,AAMT-1C9BDCF2,0x587fb5be9a055508210a623b3a66508c68345c5b,1660003200
1,True,0,2,0,1157920892373161954235709850086879000000000000...,0,1,LACNAR-84,Lachrymose Narwhal Token,0,Acentrik Asset Management,True,AAMT-563486D1,0x168c6db5e9e26d1835c7d7c03f7946a8ea973b3b,1660003200
2,True,0,5,0,1157920892373161954235709850086879000000000000...,0,0,ANTCUT-70,Antic Cuttlefish Token,0,Ocean Data NFT,True,OCEAN-NFT,0xd30dd83132f2227f114db8b90f565bca2832afbd,1660003200
3,True,0,1,0,1157920892373161954235709850086879000000000000...,0,0,ENDBAR-11,Endemic Barnacle Token,0,Acentrik Asset Management,True,AAMT-F9CE006A,0x587fb5be9a055508210a623b3a66508c68345c5b,1660003200
4,True,0,1,0,1157920892373161954235709850086879000000000000...,0,1,CONTUN-52,Contumacious Tuna Token,0,Acentrik Asset Management,True,AAMT-1C3753E9,0x168c6db5e9e26d1835c7d7c03f7946a8ea973b3b,1660003200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,True,0,1,0,100000,0,0,INTOYS-5,Intransigent Oyster Token,0,Acentrik Asset Management,True,AAMT-DA2E6CBB,0xa06711a568496e373b3c419020d8d8104efcd610,1660472967
252,True,0,2,0,1157920892373161954235709850086879000000000000...,0,0,LOQBAR-73,Loquacious Barnacle Token,0,Acentrik Asset Management,True,AAMT-799C612A,0x587fb5be9a055508210a623b3a66508c68345c5b,1660472967
253,True,0,10,0,1157920892373161954235709850086879000000000000...,0,0,ASTTUR-51,Astonishing Turtle Token,0,Acentrik Asset Management,True,AAMT-0E422E4E,0x1d35453a22ff09a88e17bb683febf3c536e9672d,1660472967
254,True,0,1,0,1157920892373161954235709850086879000000000000...,0,4,RUMCRA-42,Ruminative Crab Token,0,Acentrik Asset Management,True,AAMT-5D173A0C,0x500db43ee6966e6213ba58eaf152da593eb7432e,1660472967


In [4]:
df_fres['timestamp'].unique()
datetime.utcfromtimestamp(1660003200).strftime('%Y-%m-%d %H:%M:%S')

'2022-08-09 00:00:00'

In [16]:
print("TOTAL ORDER:")
for timestamp in df_fres['timestamp'].unique():
    dt = datetime.utcfromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M')
    total_orderCount = df_fres.loc[df_fres['timestamp'] == timestamp]['datatoken.orderCount'].astype(int).sum()
    # plt.plot(dt, total_orderCount, 'ro')
    print(f"{dt}: {total_orderCount}")

TOTAL ORDER:
2022-08-09 00:00: 42
2022-08-09 16:44: 42
2022-08-14 10:29: 52
